<a href="https://colab.research.google.com/github/hsswkwk/turbo-chainsaw/blob/feature-add-image-classification/%E7%94%BB%E5%83%8F%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像分類（Image Classification）
入力した画像情報を「形」や「模様」、「色」など特徴量に分解し、入力情報から対象物を判定できるように対応付けを行う

## CNN（Convolutional Neural Network）
畳み込みニューラルネットワーク（CNNまたはConvNet）は、畳み込みを使用しているニューラルネットワークの総称

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

# CIFAR-10データセットの読み込み
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# データの前処理（画素値を0-1の範囲に正規化）
train_images, test_images = train_images / 255.0, test_images / 255.0

# CNNモデルの構築
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

# モデルのコンパイル
# optimizer: 最適化アルゴリズム, loss: 損失関数, metrics: 評価指標
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# モデルの学習
history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

# モデルの評価
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(f'\nTest accuracy: {test_acc}')

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 77s 47ms/step - accuracy: 0.3569 - loss: 1.7421 - val_accuracy: 0.5606 - val_loss: 1.2570
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 71s 46ms/step - accuracy: 0.5721 - loss: 1.1904 - val_accuracy: 0.5864 - val_loss: 1.1666
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 85s 48ms/step - accuracy: 0.6427 - loss: 1.0116 - val_accuracy: 0.6461 - val_loss: 1.0078
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 79s 46ms/step - accuracy: 0.6841 - loss: 0.9036 - val_accuracy: 0.6638 - val_loss: 0.9597
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 82s 46ms/step - accuracy: 0.7105 - loss: 0.8265 - val_accuracy: 0.6734 - val_loss: 0.9533
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 73s 47ms/step - accuracy: 0.7344 - loss: 0.7600 - val_accuracy: 0.6871 - val_loss: 0.9083
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 74s 47ms/step - accuracy: 0.7581 - loss: 0.6959 - val_accuracy: 0.7047 - val_loss: 0.8732
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 80s 46ms/step - accuracy: 0.7738 -

## ResNet
残差ニューラルネットワーク（ResNet）は、ウェイト層が層入力を参照して残差関数を学習する深層学習モデル

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, applications

# ResNet50モデルの読み込み（ImageNetで事前学習済み）
# ImageNet:
# 物体認識ソフトウェアの研究において、ソフトウェアの最適化や性能の評価等に
# 用いるために設計された、大規模な画像データセット
resnet_model = applications.ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# CIFAR-10データセットの読み込み
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# データの前処理
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# ResNet50の出力層をCIFAR-10用に変更
# 事前学習済みのResNet50モデルに、CIFAR-10のクラス数に対応する全結合層を追加
model = models.Sequential()
model.add(resnet_model)
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))

# モデルのコンパイル
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# モデルの学習
# model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

# モデルの評価
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Test Accuracy: {accuracy}')

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 3279s 8s/step - accuracy: 0.5144 - loss: 1.6502 - val_accuracy: 0.1054 - val_loss: 3.8921
Epoch 2/10
329/391 ━━━━━━━━━━━━━━━━━━━━ 8:23 8s/step - accuracy: 0.7144 - loss: 0.8388

KeyboardInterrupt: 

## VGG16
ImageNetと呼ばれる大規模画像データセットで学習された16層(畳み込み13層、フル結合3層)からなる畳み込みニューラルネットワーク(CNN)

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, applications

# VGG16モデルの読み込み（ImageNetで事前学習済み）
vgg16_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# CIFAR-10データセットの読み込み
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# データの前処理
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# VGG16の出力層をCIFAR-10用に変更
model = models.Sequential()
model.add(vgg16_model)
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))


# モデルのコンパイル
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# モデルの学習
model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

# モデルの評価
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Test Accuracy: {accuracy}')

## Efficient Net
複合スケーリング手法を用いて、ネットワークの幅、深さ、解像度を同時に最適化し、より少ないパラメータ数で高い精度を持つCNN

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, applications

# EfficientNetB0モデルの読み込み（ImageNetで事前学習済み）
efficientnet_model = applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# CIFAR-10データセットの読み込み
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# データの前処理
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# EfficientNetB0の出力層をCIFAR-10用に変更
model = models.Sequential()
model.add(efficientnet_model)
model.add(layers.GlobalAveragePooling2D())  # GlobalAveragePooling2Dを追加
model.add(layers.Dense(10, activation='softmax'))

# モデルのコンパイル
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# モデルの学習
model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_test, y_test))

# モデルの評価
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f'Test Accuracy: {accuracy}')